In [2]:
import requests
import cv2
import os
import io
from base64 import b64encode, b64decode
from pathlib import WindowsPath
from PIL import Image
import json
import cv2
import numpy as np

In [4]:
server = "http://127.0.0.1:7860" # or "http://127.0.0.1:7861"

**Simple txt to Image**

In [5]:
class txt2Img():
    def __init__(self, query):
        self.url = server + "/sdapi/v1/txt2img"
        self.body = {
            "prompt": query["prompt"],
            "negative_prompt": "longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality",
            "seed": query["seed"],
            "subseed": -1,
            "subseed_strength": 0,
            "batch_size": 1,
            "n_iter": 1,
            "steps": query["steps"],
            "cfg_scale": 5,
            "width": query["width"],
            "height": query["height"],
            "restore_faces": False,
            "tiling": False,
            "eta": 0,
            "sampler_index": query["sampler_index"],
            "save_images": False,
            }

    def sendRequest(self):
        r = requests.post(self.url, json=self.body)
        return r.json()

def readImage(path):
    img = cv2.imread(path)
    retval, buffer = cv2.imencode('.jpg', img)
    b64img = b64encode(buffer).decode("utf-8")
    return b64img

In [13]:
query = {
        "prompt": "a cat",
        "steps": 30,
        "sampler_index": "DPM++ 2M", #Euler a
        "width": 512,
        "height": 512,
        "seed": -1,
        }

js = txt2Img(query).sendRequest()


In [22]:
b64img = js['images'][0]
image = Image.open(io.BytesIO(b64decode(b64img)))
image.save("outputs/test.jpg")

**with ControlNet**

In [18]:
class CNet2Img():
    def __init__(self,query):
        self.url = server + "/sdapi/v1/txt2img"
        self.body = {
            "prompt": query["prompt"],
            "negative_prompt": "longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality",
            "seed": query["seed"],
            "subseed": -1,
            "subseed_strength": 0,
            "batch_size": 1,
            "n_iter": 1,
            "steps": query["steps"],
            "cfg_scale": 5,
            "width": query["width"],
            "height": query["height"],
            "restore_faces": False,
            "tiling": False,
            "eta": 0,
            "sampler_index": query["sampler_index"],
            "save_images": False,
            "alwayson_scripts": {
                "controlnet": {
                        "args": [
                            {
                                "input_image": query["img"],
                                "mask": "",
                                "module": query["preprocessor"],
                                "model": query["model"],
                                "weight": 1,
                                "resize_mode": "Scale to Fit (Inner Fit)",
                                "lowvram": False,
                                "processor_res": -1,
                                "threshold_a": -1,
                                "threshold_b": -1,
                                "guidance": 1,
                                "guidance_start": 0,
                                "guidance_end": 1,
                                "guessmode": False
                            }
                        ]
                    },
            }
        }
        
    def sendRequest(self):
        r = requests.post(self.url, json=self.body)
        return r.json()

def readImage(path):
    img = cv2.imread(path)
    retval, buffer = cv2.imencode('.jpg', img)
    b64img = b64encode(buffer).decode("utf-8")
    return b64img

In [19]:
query = {
        "prompt": "a cat",
        "steps": 30,
        "sampler_index": "DPM++ 2M", #Euler a
        "width": 512,
        "height": 512,
        "seed": -1,
        "img": readImage("outputs/test.jpg"),
        "preprocessor": "depth",
        "model": "control_v11f1p_sd15_depth [cfd03158]",
        }

js = CNet2Img(query).sendRequest()


In [21]:
b64img = js['images'][0]
image = Image.open(io.BytesIO(b64decode(b64img)))
image.save("outputs/test_depth.jpg")

**only ControlNet**

In [25]:
class CNetPreprocess():
    def __init__(self,query):
        self.url = server + "/controlnet/detect"
        self.body = {
        "controlnet_module":  query["preprocessor"],
        "controlnet_input_images": [query["img"]],
        "controlnet_processor_res": 512,
        "controlnet_threshold_a": 0.5,
        "controlnet_threshold_b": 0.5,
        "low_vram": False
        }

    def sendRequest(self):
        r = requests.post(self.url, json=self.body)
        return r.json()

def readImage(path):
    img = cv2.imread(path)
    retval, buffer = cv2.imencode('.jpg', img)
    b64img = b64encode(buffer).decode("utf-8")
    return b64img

In [26]:
query = {
        "img": readImage("outputs/test.jpg"),
        "preprocessor": "depth",
        }

js = CNetPreprocess(query).sendRequest()

In [27]:
b64img = js['images'][0]
image = Image.open(io.BytesIO(b64decode(b64img)))
image.save("outputs/depth.jpg")

**Interrogate CLIP**

In [34]:
class interrogateCLIP():
    def __init__(self, img):
        self.url = server + "/sdapi/v1/interrogate"
        self.body = {
            "image": img,
            "model": "clip"
            }

    def sendRequest(self):
        r = requests.post(self.url, json=self.body)
        return r.json()

In [37]:
img = readImage("outputs/test.jpg")

js = interrogateCLIP(img).sendRequest()

In [39]:
js["caption"]

'a cat with green eyes looking off into the distance with a gray background and a black background behind it, Carlos Catasse, green eyes, a character portrait, photorealism'